In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Required Packages

In [2]:
pip install face_recognition

     |████████████████████████████████| 100.1 MB 26 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=b45da7feb672471328f355ff618d6d1905e550b6083aab7b532fab777b57d04e
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [3]:
import time, os, sys, pickle, face_recognition, cv2, sklearn
from google.colab.patches import cv2_imshow
from sklearn import svm

# Training

In [9]:

# Training the SVC classifier

# The training data would be all the face encodings from all the known images and the labels are their names
encodings = []
names = []
persons_found = []

# Training directory, here you enter the path of train directory
path = '/content/drive/MyDrive/noman_image_recognizer/train_imgs/' #Put here the path of dir that contains training data

train_dir = os.listdir(path) 

# Loop through each person in the training directory
for person in train_dir:
    pix = os.listdir(path + person)

    # Loop through each training image for the current person
    for person_img in pix:
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file(path + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image
            encodings.append(face_enc)
            # Add corresponding label
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

# Create and train the SVC classifier
clf = svm.SVC(gamma='scale')

# fit the model
clf.fit(encodings,names)

Trump/donald trump98.jpg was skipped and can't be used for training
Trump/donald trump speech129.jpg was skipped and can't be used for training
Trump/trump37.jpg was skipped and can't be used for training
Trump/donald trump speech131.jpg was skipped and can't be used for training
Trump/donald trump68.jpg was skipped and can't be used for training
Trump/donald trump85.jpg was skipped and can't be used for training
Trump/donald trump100.jpg was skipped and can't be used for training
Trump/donald trump speech123.jpg was skipped and can't be used for training
Trump/donald trump speech133.jpg was skipped and can't be used for training
Trump/trump50.jpg was skipped and can't be used for training
Bill/gates141.jpg was skipped and can't be used for training
Bill/gates32.jpg was skipped and can't be used for training
Bill/gates33.jpg was skipped and can't be used for training
Bill/gates17.jpg was skipped and can't be used for training


KeyboardInterrupt: ignored

In [6]:
#storing trained SVM Classifier into a file for future usage
file = open('/content/drive/MyDrive/noman_image_recognizer/face_classifier_upd2.model','wb') #creat a file for storing model
pickle.dump(clf,file) #storing model in newly-creasted file
file.close()

NameError: ignored

# Face Recognition on Images

This Face Recognition model is trained on following 8 Classes.

Imran Khan

Bill Gates

Elon Musk

Jeff Bezos

Jack Ma

Narendra Modi

Donald Trump

Leonardo Decaprio 


In [7]:

#Loading saved model
clf = pickle.load(open('/content/drive/MyDrive/noman_image_recognizer/face_classifier_upd2.model','rb'))

# Test directory path
test_dir_path = '/content/drive/MyDrive/noman_image_recognizer/test_imgs/'

# Path where to save the recognized images
save_dir = '/content/drive/MyDrive/noman_image_recognizer/recognized_imgs/'

# if path doesn't exist then make one
if not os.path.exists(save_dir): 
    os.mkdir(save_dir)

for img in os.listdir(test_dir_path):
    tick = time.time()
    # Load the test image with unknown faces into a numpy array
    test_image = face_recognition.load_image_file(test_dir_path+img)
    # convert an image from one color space to another
    test_image = cv2.cvtColor(test_image,cv2.COLOR_BGR2RGB)
    # print((test_image.shape))
    # Find all the faces in the test image using the default HOG-based model
    face_locations = face_recognition.face_locations(test_image)
    
    no = len(face_locations)
    print("Number of faces detected: ", no)

    # Predict all the faces in the test image using the trained classifier
    print("Found:")
    for i in range(no):
        test_image_enc = face_recognition.face_encodings(test_image,face_locations)[i]
        name = clf.predict([test_image_enc])
        print((*name))
        
        top,right,bottom,left = face_locations[i]
        
        
        # Draw a box around the face
        cv2.rectangle(test_image, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(test_image, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(test_image, *name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    cv2.imwrite(save_dir+img,test_image)
        
        
    tock = time.time()
    print('time taken: ',tock-tick,' secs')

FileNotFoundError: ignored

# Face Recognition on Video

## Video File Processing

In [ ]:
# Load the model
model = pickle.load(open('/content/drive/MyDrive/noman_image_recognizer/face_classifier_upd2.model','rb')) #model file's path

# Path to the video file
video_file = '/content/drive/MyDrive/noman_image_recognizer/sample.mp4'  #Path to video file


# Instantiate the video file
vid = cv2.VideoCapture(video_file)
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = vid.get(cv2.CAP_PROP_FPS)

tick = time.time()
output = cv2.VideoWriter(f'output_for_{video_file}.mp4', cv2.VideoWriter_fourcc(*'MP4V'),fps,(frame_width,frame_height))


# Read the video frame by frame
while (vid.isOpened()):
    ret,frame = vid.read()
    

    if ret:

        small_frame = cv2.resize(frame, (0, 0), fx=0.50, fy=0.50)
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame,cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        num_of_faces = len(face_locations)
        for i in range(num_of_faces):

            
            face_enc = face_recognition.face_encodings(rgb_small_frame)[i]
            name = model.predict([face_enc])
            
            top,right,bottom,left = face_locations[i]
            top *= 2
            right *= 2
            bottom *= 2
            left *= 2

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, *name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            tock = time.time()

        # Display the resulting image
        output.write(frame)
     #   cv2.imshow('video',frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:break
tock = time.time()

vid.release()
output.release()
cv2.destroyAllWindows()

print('time taken to process video: ',tock-tick)

## Webcam Recognition

In [ ]:

model = pickle.load(open('/content/drive/MyDrive/noman_image_recognizer/face_classifier_upd2.model','rb')) #Path to model file
vid = cv2.VideoCapture(0)




while (vid.isOpened()):
    ret,frame = vid.read()
    

    if ret:
        small_frame = cv2.resize(frame, (0, 0), fx=0.2, fy=0.20)
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame,cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        num_of_faces = len(face_locations)
        for i in range(num_of_faces):            
            face_enc = face_recognition.face_encodings(rgb_small_frame,face_locations)[i]
            name = model.predict([face_enc])
            
            top,right,bottom,left = face_locations[i]
            top *= 5
            right *= 5
            bottom *= 5
            left *= 5

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, *name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            tock = time.time()
#             print(f'took {tock-tick} time!!!')
#             sys.exit()
        # Display the resulting image
        cv2.imshow('Cam_Feed',frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:break

vid.release()
cv2.destroyAllWindows()

In [ ]:
vidcap = cv2.VideoCapture('noman_vid.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1